## The following European Classification of Individual Consumption according to Purpose (ECOICOP) data used for this example is open-source, provided by Statistics Poland

#### Link: https://github.com/UNECE/ML_dataset

In [39]:
import fasttext
import os
import pandas as pd
import numpy as np
import pickle

## Define folder locations

In [40]:
model_dir = "C:\\Users\\Justin Evans\\Documents\\Python\\UNECE\\Poland_FastText\\"
output_dir = ("C:\\Users\\Justin Evans\\Documents\\Python\\UNECE\\Poland_FastText\\")

## Define the metric values

In [41]:
# define a confidence threshold above which we will recalculate accuracy, coding rate
threshold = 0.95

# bootstrap the test file
iterations = 1000 # number of iterations run in the bootstrap
size = 0.2 # sample taken each iteration

## Load the model and predict on the test dataset

In [42]:
loaded_model = fasttext.load_model(model_dir+"model.bin")
test_data = os.path.join(os.getenv("DATADIR",""),model_dir+"test.txt")

In [43]:
loaded_model.predict("this is an example")

(('__label__032',), array([0.21888724]))

In [44]:
# load the test data for manual verification and to assess predictions
df = pd.read_csv(model_dir+"test.csv", encoding='UTF-8', dtype=str)
df.drop("Unnamed: 0",axis=1,inplace=True)

pred_classes = []
scores = []

# have the model predict on each line in the column 'formatted'
for line in df['formatted']:
    split = line.split("__label__",1)
    text = split[1]
    pred = loaded_model.predict(text)
    pred_class = pred[0][0].replace("__label__","")
    pred_score = pred[1][0]
    pred_classes.append(pred_class)
    scores.append(pred_score)
    
df['pred'] = pred_classes
df['score'] = scores
   
# add the code_key back
with open("code_key.txt", "rb") as file:
    code_dict = pickle.load(file)
    inv_map = {v: k for k, v in code_dict.items()} 
df['code_text_pred'] = df.pred.map(inv_map)
    
df.head()

,text,code_text,code,formatted,pred,score,code_text_pred
0,filets de maquereaux a l&#39;huile vegetale et,Autres préparations de poisson et de fruits de...,008,__label__008 filets de maquereaux a l&#39;huil...,008,0.985303,Autres préparations de poisson et de fruits de...
1,grand glace aux graines de pavot d&#39;or au c...,crème glacée,053,__label__053 grand glace aux graines de pavot ...,053,0.965585,crème glacée
2,kabanos de saucisses polonaises a la volaille ...,viandes,059,__label__059 kabanos de saucisses polonaises a...,059,0.996574,viandes
3,salade de legumes cuisine polonaise,Légumes secs et autres préparations à base de ...,033,__label__033 salade de legumes cuisine polonaise,033,0.914373,Légumes secs et autres préparations à base de ...
4,dessert corse chocolat pour enrober la pate 64...,chocolat,052,__label__052 dessert corse chocolat pour enrob...,052,0.672030,chocolat


## Evaluation Metrics

In [45]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import resample
from pandas import DataFrame

In [46]:
# create lists to for evaluation metrics

# fix column types
df["pred"] = df["pred"].astype(str)
df["code_text_pred"] = df["code_text_pred"].astype(str)

# convert df to list
list_actual = df["code_text"].tolist()
list_predicted = df["code_text_pred"].tolist()

In [47]:
# basic metrics for accuracy 
df_threshold = df[df['score'] > threshold] # create a dataframe with only predictions above the defined threshold

overall_acc = round(accuracy_score(df.code_text, df.code_text_pred)*100,2)
accuracy = round(accuracy_score(df_threshold.code_text, df_threshold.code_text_pred)*100,2)
codingrate = round((df_threshold.shape[0]/df.shape[0])*100,2)

print("Overall Accuracy:",overall_acc)
print("Threshold Applied:", threshold)
print("Accuracy:",accuracy)
print("Coding Rate:",codingrate)

Overall Accuracy: 97.08
Threshold Applied: 0.95
Accuracy: 99.35
Coding Rate: 71.99


In [48]:
# classification report

report = classification_report(list_actual, list_predicted, output_dict=True)
df_class = pd.DataFrame(report).transpose()
df_class["class"] = df_class.index
df_class = df_class.reset_index(drop=True)
df_class.to_csv(output_dir + "classification_report.csv")

df_class.head(10)

,f1-score,precision,recall,support,class
0,0.909091,1.000000,0.833333,6.0,Autres graisses animales
1,1.000000,1.000000,1.000000,21.0,Autres huiles comestibles
2,0.833333,1.000000,0.714286,7.0,Autres légumes à légumes et préparations à bas...
3,0.953488,0.959064,0.947977,173.0,Autres produits alimentaires nca
4,0.914286,0.888889,0.941176,17.0,Autres produits céréaliers
5,0.966981,0.966981,0.966981,212.0,Autres produits de boulangerie
6,0.993197,0.986486,1.000000,73.0,Autres produits laitiers
7,0.970588,0.961165,0.980198,101.0,Autres préparations de poisson et de fruits de...
8,0.956522,0.936170,0.977778,45.0,Autres préparations de viande
9,0.857143,1.000000,0.750000,4.0,Autres viandes


In [49]:
# overall F1, precision, recall metrics are included in the classification report
f1_weighted_avg = df_class.iloc[-1,0]
precision_weighted_avg = df_class.iloc[-1,0]
recall_weighted_avg = df_class.iloc[-1,2]

f1_macro_avg = df_class.iloc[-2,0]
precision_macro_avg = df_class.iloc[-2,0]
recall_macro_avg = df_class.iloc[-2,2]

# create a df to show data
df_metrics = df_class.iloc[[-1,-2,-3]]
df_metrics.head()

,f1-score,precision,recall,support,class
63,0.970608,0.971212,0.970760,3420.0,weighted avg
62,0.955313,0.965621,0.950552,3420.0,macro avg
61,0.970760,0.970760,0.970760,3420.0,micro avg


In [50]:
# bootstrap - source: machinelearningmastery.com/calculate-bootstrap-confidence-intervals-machine-learning-results-python/

def bootstrap(df):
    # run bootstrap
    n_iterations = iterations
    n_size = int(len(df) * float(size))
    stats = list()

    for i in range(n_iterations):
        boot = resample(df, n_samples=n_size)
        code = boot.code_text
        pred = boot.code_text_pred
        score = accuracy_score(code, pred)
        stats.append(score)

    # confidence intervals
    alpha = 0.95
    p = ((1.0 - alpha) / 2.0) * 100
    lower = max(0.0, np.percentile(stats, p))
    p = (alpha + ((1.0 - alpha) / 2.0)) * 100
    upper = min(1.0, np.percentile(stats, p))
    
    print('%.1f confidence interval %.1f%% and %.1f%%' % (alpha * 100, lower * 100, upper * 100))

bootstrap(df)

95.0 confidence interval 95.6% and 98.2%


## Export Metrics

In [51]:
# produce a report with model evaluation metrics
report = open(model_dir+"MODEL_METRICS.txt","w")
lines = ["Overall Accuracy: "+str(overall_acc)+"\n",
        "Threshold Applied: "+str(threshold)+"\n", 
        "Accuracy: "+str(accuracy)+"\n", 
        "Coding Rate:"+str(codingrate)+"\n"+"\n", 
         
         
        'Type: '+str('Weighted Average')+", "+str('Macro Average')+"\n",
        'F1_score: '+str(f1_weighted_avg)+", "+str(f1_macro_avg)+"\n",         
        'Precision: '+str(precision_weighted_avg)+", "+str(precision_macro_avg)+"\n",      
        'Recall: '+str(recall_weighted_avg)+", "+str(recall_macro_avg)+"\n",      
        ]
report.writelines(lines) 
report.close()

In [52]:
print('done')

done
